# Quick-start: Set up your first equation
This is a jupyter notebook serving as a quick/first interactive tutorial for the usage of *phyem*.

To let Python know where to find the *phyem* package, we need to add the dir that contains the package to the searching path of Python unless it is already in a system path.

In [1]:
import sys
path_to_phyem = '../../../../'   # the path to dir that containing the phyem package.
sys.path.append(path_to_phyem)
import phyem as ph  # import the phyem package
ph.config._set_matplot_block(False)

We now set the dimensions of the embedding space to be 3.

In [2]:
ph.config.set_embedding_space_dim(3)

Initialize an abstract manifold by

In [3]:
manifold = ph.manifold(3)

Then an abstract mesh is built upon manifold `manifold`.

In [4]:
mesh = ph.mesh(manifold)

Using function `list_meshes` of *phyem* can list all existing meshes in the current console.

In [5]:
ph.list_meshes()


 Existing meshes:
---------------- symbolic - <manifold> -------------------------
             \mathfrak{M} | <Manifold \mathcal{M} at 0x000002B6FF93E8D0>


where we can see the symbolic representation of the mesh and the manifold on which it is built. If we render the symbolic representation of the mesh with an equation environment, we see $\mathfrak{M}$.

We then can set spaces upon this mesh. For example, we set up spaces $\Lambda^{(0)}(\mathcal{M})$, $\Lambda^{(1)}(\mathcal{M})$, $\Lambda^{(2)}(\mathcal{M})$, $\Lambda^{(3)}(\mathcal{M})$, i.e., the Hilbert spaces of scalar valued 0- to 3-forms, by

In [6]:
ph.space.set_mesh(mesh)
O0 = ph.space.new('Lambda', 0)  # Lambda is the indicator for (standard) scalar valued form spaces.
O1 = ph.space.new('Lambda', 1)
O2 = ph.space.new('Lambda', 2)
O3 = ph.space.new('Lambda', 3)
ph.list_spaces()  # we can also list all existing spaces


 Implemented spaces:
   abbreviation - description
         Lambda | scalar valued k-form space

 Existing spaces:
        On mesh \mathfrak{M}
              0: \widetilde{\Lambda}^{(0)}(\mathcal{M})
              1: \widetilde{\Lambda}^{(1)}(\mathcal{M})
              2: \widetilde{\Lambda}^{(2)}(\mathcal{M})
              3: \widetilde{\Lambda}^{(3)}(\mathcal{M})


where we see first a list of all implemented spaces and then the existing spaces till this moment. 

A form is just a instance of such space. So we make forms from spaces by calling method `make_form` which takes two arguments representing the symbolic representation and the linguistic representation of the form. These forms are the root forms.

In [7]:
w = O1.make_form(r'\omega^1', "vorticity1") 
u = O2.make_form(r'u^2', r"velocity2")
f = O2.make_form(r'f^2', r"body-force")
P = O3.make_form(r'P^3', r"total-pressure3")
ph.list_forms(locals())  # this will generate a table in a separete figure showing.

<Figure size 1600x500 with 1 Axes>

where we have used function `list_forms` to visualize/list the exsiting forms. Note that, when forms are not many, a separete plot will be generated where the forms are visualized in a table. As this process invokes *matplotlib* and *latex*, make sure they are installed. When the forms are many, instead of plotting, a text list will appear in the default console. 

> The argument `locals()` restricts `list_forms` function to only the local variables. Since intermediate forms may be generated during some processes, if we do not restrict the range of `list_forms`, they may also be shown. But at this moment, `ph.list_forms(locals())` and `ph.list_forms()` make no differences.  

> When it is the first time to invoke *matplotlib* and *latex*, it may be very slow since there are large amount of interplays among the packages. Be patient. Things become much better later on.

We can also visualize a particular form by calling its `pr` method. For example,

In [8]:
u.pr()

<Figure size 1200x600 with 1 Axes>

Furthermore, we can use these root forms to build other forms through operators like $\wedge$, $\star$, $\mathrm{d}^\ast$, $\mathrm{d}$, $\partial_t$ and so on.

In [9]:
wXu = w.wedge(ph.Hodge(u))
dsP = ph.codifferential(P)
dsu = ph.codifferential(u)
du = ph.d(u)
du_dt = ph.time_derivative(u)
ph.list_forms(locals())

<Figure size 1600x1000 with 1 Axes>

Now, if you try `ph.list_forms()` which does not restrict the range of `list_forms` function to the local environment, the outputs are different.

In [10]:
ph.list_forms()

---> 2984995829648 | \omega^1 = \textsf{vorticity1}
---> 2984976884240 | u^2 = \textsf{velocity2}
---> 2984995829840 | f^2 = \textsf{body-force}
---> 2984995830224 | P^3 = \textsf{total-pressure3}
---> 2983697562640 | {\star}u^2 = $\star$ \textsf{velocity2}
---> 2983697455056 | \omega^1{\wedge}\left({\star}u^2\right) = \textsf{vorticity1} $\wedge$ \{$\star$ \textsf{velocity2}\}
---> 2984998045072 | \mathrm{d}^{\ast}P^3 = $\mathrm{d}^{\ast}$ \textsf{total-pressure3}
---> 2984997506000 | \mathrm{d}^{\ast}u^2 = $\mathrm{d}^{\ast}$ \textsf{velocity2}
---> 2983697618448 | \mathrm{d}u^2 = $\mathrm{d}$ \textsf{velocity2}
---> 2983697619920 | \partial_{t}u^2 = $\partial_{t}$ \textsf{velocity2}


Basically, we see the `id` and then *symbolic representation* = *linguistic representation* of all forms.

With forms we can construct equations (usually **partial differential equations, PDEs**) through function `ph.pde`.

In [11]:
exp1 = [
    'dudt + wXu - dsP = f',
    'w = dsu',
    'du = 0',
]
itp = {
    'dudt': du_dt,
    'wXu': wXu,
    'dsP': dsP,
    'f': f,
    'w': w,
    'dsu': dsu,
    'du': du,
}
pde1 = ph.pde(exp1, itp)
pde1.unknowns = [u, w, P]

where we send an expression (`exp1`) and an interpreter (`itp`) to `ph.pde` to initialize an equation object named `pde1`. You can see that in `exp1` we use string to represent the variables, terms and operators. The interpreter, `itp`, inteprets the string representations and thus `ph.pde` knows to use correct ingredients.

You can avoid defining the interpreter manually by use the built-in function `locals`. For example,

In [12]:
exp2 = [
    'du_dt + wXu - dsP = f',
    'w = dsu',
    'du = 0',
]
pde2 = ph.pde(exp2, locals())
pde2.unknowns = [u, w, P]

In this way, you lose the freedom of naming the terms in the expression because `locals()` gives a dictionary whose keys are exactly the vraible names in the local environment. See `'dudt'` in `exp1` and `du_dt` in `exp2`.

After constructing our equations, we may want to have a close look at it to check if any mistakes we made. We can call method `pr`, standing for *print representation*, to do that. For example,

In [13]:
pde2.pr(indexing=False)

<Figure size 800x600 with 1 Axes>